# OpenAI Function Calling In LangChain

In [1]:
import os
import openai

from dotenv import load_dotenv
load_dotenv(".env", override=True)# read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. 

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like this:

In [3]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [4]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [5]:
foo.name

'Joe'

In [6]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [7]:
foo.age

'bar'

In [8]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [9]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [10]:
foo_p.name

'Jane'

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next line is expected to fail.</p>

In [11]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing

In [12]:
class Class(BaseModel):
    students: List[pUser]

In [13]:
obj = Class(students=[pUser(name="Jane", age=32, email="jane@gmail.com")])

In [14]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition


In [15]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [18]:
# from langchain.utils.openai_functions import convert_pydantic_to_openai_function

from langchain_core.utils.function_calling import convert_to_openai_function

In [21]:
weather_function = convert_to_openai_function(WeatherSearch)

In [22]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [23]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next cell is expected to generate an error.</p>

In [24]:
convert_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [25]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [26]:
convert_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [27]:
from langchain_openai import ChatOpenAI

In [28]:
model = ChatOpenAI(model = "gpt-5-nano")

In [29]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 154, 'prompt_tokens': 149, 'total_tokens': 303, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--c5babcb8-ffd8-4fba-ab14-44f872ffc0c2-0', usage_metadata={'input_tokens': 149, 'output_tokens': 154, 'total_tokens': 303, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 128}})

In [30]:
model_with_function = model.bind(functions=[weather_function])

In [31]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 154, 'prompt_tokens': 148, 'total_tokens': 302, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--7da2069a-3856-4740-8f69-fd2c53be7462-0', usage_metadata={'input_tokens': 148, 'output_tokens': 154, 'total_tokens': 302, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 128}})

## Forcing it to use a function

We can force the model to use a function

In [32]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [33]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 282, 'prompt_tokens': 148, 'total_tokens': 430, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--0d0d94ba-8e56-465d-b2b8-f0333931dba9-0', usage_metadata={'input_tokens': 148, 'output_tokens': 282, 'total_tokens': 430, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 256}})

In [34]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"LAX"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 410, 'prompt_tokens': 143, 'total_tokens': 553, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--73ac73e0-1367-4a25-8a3c-094ec5951595-0', usage_metadata={'input_tokens': 143, 'output_tokens': 410, 'total_tokens': 553, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 384}})

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [35]:
from langchain.prompts import ChatPromptTemplate

In [36]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [37]:
chain = prompt | model_with_function

In [38]:
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 154, 'prompt_tokens': 157, 'total_tokens': 311, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--1fd8fb0b-1dab-42b5-8cb7-7be062c095a2-0', usage_metadata={'input_tokens': 157, 'output_tokens': 154, 'total_tokens': 311, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 128}})

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [39]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [40]:
functions = [
    convert_to_openai_function(WeatherSearch),
    convert_to_openai_function(ArtistSearch),
]

In [41]:
model_with_functions = model.bind(functions=functions)

In [42]:
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='Do you mean San Francisco International Airport (SFO) or another airport near San Francisco (e.g., OAK, SJC)? If you want, I can fetch the current weather for SFO by default—just say “SFO” or tell me which airport code you prefer.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 707, 'prompt_tokens': 192, 'total_tokens': 899, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 640, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--750c5406-ddcd-4e7d-aedc-38fddd73379b-0', usage_metadata={'input_tokens': 192, 'output_tokens': 707, 'total_tokens': 899, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 640}})

In [43]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='- Love Story\n- Blank Space\n- Shake It Off\n\nWant alternatives from a specific era? I can suggest more.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 354, 'prompt_tokens': 194, 'total_tokens': 548, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--ff21edaa-a9ce-4568-91f8-bc43d4b3c660-0', usage_metadata={'input_tokens': 194, 'output_tokens': 354, 'total_tokens': 548, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 320}})

In [44]:
model_with_functions.invoke("hi!")

AIMessage(content='Hi there! How can I help today? I can:\n- Check the weather at a specific airport (just give me the 3-letter code, like JFK or LAX)\n- Look up songs by a given artist (tell me the artist and how many results you want)\n- Answer questions or help with other requests\n\nWhat would you like to do?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 401, 'prompt_tokens': 187, 'total_tokens': 588, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--614d0512-4c74-495e-b7bf-93662dc12ee9-0', usage_metadata={'input_tokens': 187, 'output_tokens': 401, 'total_tokens': 588, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'r